<a href="https://colab.research.google.com/github/MarkZuck10/economic-recession-prediction/blob/main/BERTweet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. preparing dataset

In [1]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/2023_02_01_rawtext.csv')
df.shape

(21954, 2)

In [2]:
df.isna().sum().sort_values(ascending=False)

,0
text,0
tweetid,0


In [3]:
df.duplicated().sum()

0

# 2. batch processed pretrained BERTweet

In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
!pip install emoji==0.6.0

In [5]:
# Loading the pre-trained BERTweet sentiment analysis model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("finiteautomata/bertweet-base-sentiment-analysis")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
def preprocess_tweets(tweets):
    return tokenizer(tweets, padding=True, truncation=True, return_tensors="pt")

batch_size = 256
sentiment_labels = ["Negative", "Neutral", "Positive"]
predicted_sentiments = []

In [7]:
for i in range(0, len(df), batch_size):
    batch_tweets = df['text'][i:i+batch_size].tolist()
    encoded_tweets = preprocess_tweets(batch_tweets)

    with torch.no_grad():
        outputs = model(**encoded_tweets)
        predictions = torch.argmax(outputs.logits, dim=1)

    batch_predicted_sentiments = [sentiment_labels[label.item()] for label in predictions]
    predicted_sentiments.extend(batch_predicted_sentiments)

In [8]:
df['bertweet_sentiment'] = predicted_sentiments

# 3. Saving dataset

In [9]:
sentiment_counts = df['bertweet_sentiment'].value_counts()
print(sentiment_counts)

bertweet_sentiment
Neutral     10165
Negative     8739
Positive     3050
Name: count, dtype: int64


In [10]:
df.to_csv('/content/drive/My Drive/01-02-23_BERTweet-sentiment.csv', index=False)